In [ ]:
# comment this block and use as per your pc, this is for windows
import findspark
findspark.init("C:/spark") 

from pyspark import SparkContext
sc=SparkContext("local","Pyspark new")

In [2]:
import os
import findspark

os.environ["SPARK_HOME"] = "//home/hadoop/spark-3.5.6-bin-hadoop3"  # your Spark install path
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.11.0-openjdk-amd64"

findspark.init()

In [3]:
from pyspark import SparkContext
sc=SparkContext("local","Pyspark new")

25/10/03 14:46:59 WARN Utils: Your hostname, vikasan-Strix-G51RC resolves to a loopback address: 127.0.1.1; using 10.52.10.14 instead (on interface wlo1)
25/10/03 14:46:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/03 14:46:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
print("Spark Version:", spark.version)

Spark Version: 3.5.6


# PHASE 1- DATASET UNDERSTANDING AND SETUP

#### Vikasan's part

In [5]:
from pyspark.sql.functions import * 

In [ ]:
clickstream_df = spark.read \
    .option("delimiter", "\t") \
    .option("header",False)\
    .option("InferSchema",True)\
    #.csv("file:///home/hadoop/Desktop/BDA_miniproj/dataset/clickstream-enwiki-2025-07.tsv") #in linux
    #.csv(r"dataset\clickstream-enwiki-2025-07.tsv")    #windows

In [13]:
columns = ["prev", "curr", "type", "n"]
clickstream_df = clickstream_df.toDF(*columns)  # writing the column headers

# show few rows
clickstream_df.show(10)

+-------------------+--------------------+--------+---------+
|               prev|                curr|    type|        n|
+-------------------+--------------------+--------+---------+
|        other-empty|           Main_Page|external|167194874|
|Meteor_(miniseries)|     Ernie_Barbarash|    link|       13|
|       other-search|       Ozzy_Osbourne|external|  7956732|
|   Meteor_(missile)|                Euro|    link|       13|
|        other-empty|        Hyphen-minus|external|  7819580|
|   Meteor_(missile)|      Royal_Ordnance|    link|       13|
|     other-internal|           Main_Page|external|  6902577|
|   Meteor_(missile)|Saab_Bofors_Dynamics|    link|       13|
|       other-search|Superman_(2025_film)|external|  5462456|
|   Meteor_(missile)|     Sukhoi_Su-30MKI|    link|       13|
+-------------------+--------------------+--------+---------+
only showing top 10 rows



In [14]:
clickstream_df.columns

['prev', 'curr', 'type', 'n']

In [15]:
noOfRows= clickstream_df.count()

In [16]:
noOfRows

36131419

#### Prajwal's part

In [9]:
# do your part here 

In [17]:
# check the schema
clickstream_df.printSchema()
clickstream_df.show(5, truncate=False)

root
 |-- prev: string (nullable = true)
 |-- curr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- n: integer (nullable = true)

+-------------------+---------------+--------+---------+
|prev               |curr           |type    |n        |
+-------------------+---------------+--------+---------+
|other-empty        |Main_Page      |external|167194874|
|Meteor_(miniseries)|Ernie_Barbarash|link    |13       |
|other-search       |Ozzy_Osbourne  |external|7956732  |
|Meteor_(missile)   |Euro           |link    |13       |
|other-empty        |Hyphen-minus   |external|7819580  |
+-------------------+---------------+--------+---------+
only showing top 5 rows



In [18]:
# checking nulls
missing_counts = clickstream_df.select([
    count(when(col(c).isNull(), c)).alias(c) for c in clickstream_df.columns
])
missing_counts.show()

+----+----+----+---+
|prev|curr|type|  n|
+----+----+----+---+
|   0|   0|   0|  0|
+----+----+----+---+



In [19]:
stats = clickstream_df.select(
    min("n").alias("minOf_n"),
    max("n").alias("maxOf_n"),
    mean("n").alias("meanOf_n")
)


In [20]:
stats.show()

+-------+---------+------------------+
|minOf_n|  maxOf_n|          meanOf_n|
+-------+---------+------------------+
|     10|167194874|181.54733773395392|
+-------+---------+------------------+



In [21]:
a=clickstream_df.orderBy(col("n").asc()).select("prev","curr","n").limit(1)
b=clickstream_df.orderBy(col("n").desc()).select("prev","curr","n").limit(1)

In [22]:
a.show()
b.show()

+----+------------------+---+
|prev|              curr|  n|
+----+------------------+---+
|  !!|!_(disambiguation)| 10|
+----+------------------+---+



+-----------+---------+---------+
|       prev|     curr|        n|
+-----------+---------+---------+
|other-empty|Main_Page|167194874|
+-----------+---------+---------+



#### Dhruva's part

In [14]:
# do your part here 
# inspect curr, prev, link (distribution analysis)

In [23]:
# Find rows where n is not a pure number
df_invalid_n = clickstream_df.filter(~col("n").cast("string").rlike("^[0-9]+$"))
df_invalid_n.show(20, truncate=False)

# Count them
print("Invalid n rows:", df_invalid_n.count())



+----+----+----+---+
|prev|curr|type|n  |
+----+----+----+---+
+----+----+----+---+



Invalid n rows: 0


In [24]:
#inspect 'type column'
clickstream_df.groupBy("type").count().show(truncate=False)

+--------+--------+
|type    |count   |
+--------+--------+
|link    |22940461|
|other   |790412  |
|external|12400546|
+--------+--------+



In [ ]:
# inspect 'prev' column
clickstream_df.groupBy("prev").count().orderBy(desc("count")).show(20, truncate=False)        

+-------------------------------------------------------------+-------+
|prev                                                         |count  |
+-------------------------------------------------------------+-------+
|other-empty                                                  |5696646|
|other-search                                                 |4075166|
|other-internal                                               |1785075|
|other-external                                               |642457 |
|Main_Page                                                    |263151 |
|other-other                                                  |201202 |
|Deaths_in_2025                                               |5616   |
|Wikipedia                                                    |3684   |
|Wiki                                                         |2743   |
|List_of_United_States_counties_and_county_equivalents        |2480   |
|List_of_American_films_of_2025                               |2

# PHASE 2 - DATA PREPROCESSING

In [ ]:
from pyspark.sql.functions import *

#### Vikasan's part
- we are dropping the rows that have 'prev' = 'other -other' as it doesn’t give meaningful context about where the user came from.

In [31]:
# Remove rows where 'prev' is 'other-empty'
other_prev_col = clickstream_df.filter(col('prev') == 'other-other')

In [33]:
other_prev_col.count()

201202

In [34]:
clickstream_df = clickstream_df.filter(col('prev') != 'other-other')

In [36]:
print("After dropping 'other-other' rows from prev column: \n")
clickstream_df.show()

After dropping 'other-other' rows from prev column: 

+-------------------+--------------------+--------+---------+
|               prev|                curr|    type|        n|
+-------------------+--------------------+--------+---------+
|        other-empty|           Main_Page|external|167194874|
|Meteor_(miniseries)|     Ernie_Barbarash|    link|       13|
|       other-search|       Ozzy_Osbourne|external|  7956732|
|   Meteor_(missile)|                Euro|    link|       13|
|        other-empty|        Hyphen-minus|external|  7819580|
|   Meteor_(missile)|      Royal_Ordnance|    link|       13|
|     other-internal|           Main_Page|external|  6902577|
|   Meteor_(missile)|Saab_Bofors_Dynamics|    link|       13|
|       other-search|Superman_(2025_film)|external|  5462456|
|   Meteor_(missile)|     Sukhoi_Su-30MKI|    link|       13|
|       other-search|          Hulk_Hogan|external|  4169217|
|    Meteor_Airlines|             Tinghir|    link|       13|
|       other-se

In [37]:
clickstream_df.count()

35930217

In [ ]:
#clickstream_df = clickstream_df.dropna() #No nulls

In [45]:
#clickstream_df.write.parquet("file:///home/hadoop/Desktop/BDA_miniproj/dataset/cleaned_clickstream.parquet")

#### Prajwal's part

#### Dhruva's part